In [1]:
import pandas as pd
import wget
import duckdb

filename = 'playerstatistics.csv'
lookup_file = 'nba_player_lookup.csv'
#wget.download(f'https://storage.googleapis.com/nba_award_predictor/nba_data/{filename}')
wget.download(f'https://storage.googleapis.com/nba_award_predictor/nba_data/{lookup_file}')
playerstatistics_df = pd.read_csv(filename)
nba_lookup_df = pd.read_csv(lookup_file)

query = """

SELECT DISTINCT firstname, lastname, CONCAT(firstname,' ',lastname) AS full_name, personid, SUM(numMinutes) AS total_minutes
FROM playerstatistics_df
WHERE gameType NOT IN ('Preseason')
GROUP BY firstname, lastname, personid
HAVING SUM(numMinutes) > 0
ORDER BY SUM(numMinutes) DESC


"""

player_statistics_result_df = duckdb.query(query).df()
player_statistics_result_df


/tmp/ipykernel_6137/927932734.py:9: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  playerstatistics_df = pd.read_csv(filename)


,firstName,lastName,full_name,personId,total_minutes
0,LeBron,James,LeBron James,2544,70341.95
1,Kareem,Abdul-Jabbar,Kareem Abdul-Jabbar,76003,64883.00
2,Karl,Malone,Karl Malone,252,62437.00
3,Kobe,Bryant,Kobe Bryant,977,56505.00
4,Dirk,Nowitzki,Dirk Nowitzki,1717,56479.00
...,...,...,...,...,...
4643,Cedric,Hunter,Cedric Hunter,77083,1.00
4644,Mark,Baker,Mark Baker,1858,1.00
4645,Barry,Sumpter,Barry Sumpter,78654,1.00
4646,Norm,Grekin,Norm Grekin,76886,1.00


In [2]:
query = """

SELECT * FROM nba_lookup_df
"""

nba_lookup_df = duckdb.query(query).df()
nba_lookup_df

,player_id,player_name,first_name,last_name,is_active
0,76001,Alaa Abdelnaby,Alaa,Abdelnaby,False
1,76002,Zaid Abdul-Aziz,Zaid,Abdul-Aziz,False
2,76003,Kareem Abdul-Jabbar,Kareem,Abdul-Jabbar,False
3,51,Mahmoud Abdul-Rauf,Mahmoud,Abdul-Rauf,False
4,1505,Tariq Abdul-Wahad,Tariq,Abdul-Wahad,False
...,...,...,...,...,...
5130,78650,Matt Zunic,Matt,Zunic,False
5131,1641783,Tristan da Silva,Tristan,da Silva,True
5132,1628427,Vlatko Čančar,Vlatko,Čančar,True
5133,1642365,Nikola Đurišić,Nikola,Đurišić,False


In [3]:
import unicodedata
import re

def remove_accents(text):
    """
    Remove accent marks from input text while preserving the base characters.
    Also handles special characters like Đ/đ.
    
    Example:
    "Nikola Đurišić" -> "Nikola Durisic"
    """
    # First, handle special characters that need specific replacements
    special_chars = {
        'Đ': 'D', 'đ': 'd',  # Serbian/Croatian D with stroke
        'Ł': 'L', 'ł': 'l',  # Polish L with stroke
        'Ø': 'O', 'ø': 'o',  # Danish/Norwegian O with stroke
        'Ŧ': 'T', 'ŧ': 't',  # Sami T with stroke
        'Æ': 'AE', 'æ': 'ae',  # Æ/æ ligature
        'Œ': 'OE', 'œ': 'oe',  # Œ/œ ligature
        'ß': 'ss',  # German eszett
    }
    
    for char, replacement in special_chars.items():
        text = text.replace(char, replacement)
    
    # Normalize the text to decompose characters into base character and accent mark
    normalized_text = unicodedata.normalize('NFKD', text)
    
    # Filter out the non-spacing marks (accent marks)
    result = ''.join(c for c in normalized_text if not unicodedata.category(c).startswith('Mn'))
    
    return result

# Example
name = "Nikola Đurišić"
name = 'Kristaps Porziņģis'
clean_name = remove_accents(name)
print(f"Original: {name}")
print(f"Without accents: {clean_name}")

Original: Kristaps Porziņģis
Without accents: Kristaps Porzingis


In [4]:
nba_lookup_df['player_name'] = nba_lookup_df['player_name'].apply(remove_accents)
player_statistics_result_df['full_name'] = player_statistics_result_df['full_name'].apply(remove_accents)
nba_lookup_df_cleaned = nba_lookup_df
player_statistics_result_df_cleaned = player_statistics_result_df

In [5]:
query = """
WITH CTE AS (
SELECT * FROM player_statistics_result_df_cleaned a
JOIN nba_lookup_df_cleaned b
ON a.full_name = b.player_name
)

SELECT DISTINCT full_name
FROM CTE

"""

players_with_matching_lookup_df = duckdb.query(query).df()
players_with_matching_lookup_df

,full_name
0,Tariq Abdul-Wahad
1,Shareef Abdur-Rahim
2,Tom Abernethy
3,Alex Abrines
4,Alex Acker
...,...
4583,Ante Zizic
4584,Jim Zoet
4585,Tristan da Silva
4586,Vlatko Cancar


In [6]:
query = """

SELECT DISTINCT full_name
FROM player_statistics_result_df_cleaned

"""

all_players_with_stats_df = duckdb.query(query).df()
all_players_with_stats_df

,full_name
0,Dirk Nowitzki
1,Tim Duncan
2,Jason Kidd
3,Kevin Garnett
4,Wilt Chamberlain
...,...
4601,Gani Lawal
4602,Forest Able
4603,Trey McKinney-Jones
4604,Michael Foster Jr.


In [7]:
query = """

SELECT * FROM all_players_with_stats_df
EXCEPT
SELECT * FROM players_with_matching_lookup_df

"""

names_that_need_mapping_df = duckdb.query(query).df()
names_that_need_mapping_df

,full_name
0,Nene Hilario
1,Jianlian Yi
2,Brandon Boston Jr.
3,Nigel Hayes
4,Chaundee Brown
5,Yue Sun
6,Cui Yongxi
7,Jimmy Butler
8,Reggie Bullock
9,Kenyon Martin Jr.


In [8]:
# query = """
# WITH CTE AS (
# SELECT * FROM player_statistics_result_df a
# JOIN nba_lookup_df b
# ON a.full_name = b.player_name
# )

# SELECT DISTINCT full_name
# FROM player_statistics_result_df
# EXCEPT
# SELECT DISTINCT full_name
# FROM CTE

# """

# names_to_map = duckdb.query(query).df()
# names_to_map

In [11]:
query = """

SELECT *, 'nba_api_lookup' AS data_source
--FROM common_player_info_df
FROM nba_lookup_df
WHERE 1=1
AND last_name LIKE '%Dunleavy%'
--AND first_name LIKE '%Robert%'

"""

output = duckdb.query(query).df()
output

,player_id,player_name,first_name,last_name,is_active,data_source
0,2399,Mike Dunleavy,Mike,Dunleavy,False,nba_api_lookup
1,76616,Mike Dunleavy,Mike,Dunleavy,False,nba_api_lookup


In [10]:
import pandas as pd

# Create a list of dictionaries where each dictionary represents one row
data = [
    {"in_table_name": "Jimmy Butler", "nba_lookup_name": "Jimmy Butler III"},
    {"in_table_name": "J.D. Davison", "nba_lookup_name": "JD Davison"},
    {"in_table_name": "Ha Ha", "nba_lookup_name": "Ha Seung-jin"},
    {"in_table_name": "Bear, The Body Hoffman", "nba_lookup_name": "Paul 'The Bear' Hoffman"},
    {"in_table_name": "Kenyon Martin Jr.", "nba_lookup_name": "KJ Martin"},
    {"in_table_name": "Ted, Hound Dog McClain", "nba_lookup_name": "Ted 'Hound Dog' McClain"},
    {"in_table_name": "Tre Scott", "nba_lookup_name": "Trevon Scott"},
    {"in_table_name": "Sun Sun", "nba_lookup_name": "Sun Yue"},
    {"in_table_name": "O.G. Anunoby", "nba_lookup_name": "OG Anunoby"},
    {"in_table_name": "Brandon Boston Jr.", "nba_lookup_name": "Brandon Boston"},
    {"in_table_name": "Chaundee Brown", "nba_lookup_name": "Chaundee Brown Jr."},
    {"in_table_name": "Nigel Hayes", "nba_lookup_name": "Nigel Hayes-Davis"},
    {"in_table_name": "RJ Nembhard", "nba_lookup_name": "Ruben Nembhard Jr."},
    {"in_table_name": "D.J. Stewart", "nba_lookup_name": "DJ Stewart"},
    {"in_table_name": "Jianlian Yi", "nba_lookup_name": "Yi Jianlian"},
    {"in_table_name": "Yue Sun", "nba_lookup_name": "Sun Yue"},
    {"in_table_name": "Reggie Bullock", "nba_lookup_name": "Reggie Bullock Jr."},
    {"in_table_name": "ga Bitadze", "nba_lookup_name": "Goga Bitadze"},
    {"in_table_name": "Seung-Jin Ha", "nba_lookup_name": "Ha Seung-jin"},
    {"in_table_name": "Matthew Hurt", "nba_lookup_name": "Matt Hurt"},
    {"in_table_name": "RJ Nembhard Jr.", "nba_lookup_name": "Ruben Nembhard Jr."},    
]

name_mapping_df = pd.DataFrame(data)
name_mapping_df

,in_table_name,nba_lookup_name
0,Jimmy Butler,Jimmy Butler III
1,J.D. Davison,JD Davison
2,Ha Ha,Ha Seung-jin
3,"Bear, The Body Hoffman",Paul 'The Bear' Hoffman
4,Kenyon Martin Jr.,KJ Martin
5,"Ted, Hound Dog McClain",Ted 'Hound Dog' McClain
6,Tre Scott,Trevon Scott
7,Sun Sun,Sun Yue
8,O.G. Anunoby,OG Anunoby
9,Brandon Boston Jr.,Brandon Boston
